In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='citeseer',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_2',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [3]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [4]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [5]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([4230, 4230]) torch.Size([4230, 4230])
Training...
Epoch: 0001 train_loss= 1.38637 time= 0.10810
Epoch: 0002 train_loss= 1.38563 time= 0.10687
Epoch: 0003 train_loss= 1.38410 time= 0.10397
Epoch: 0004 train_loss= 1.38187 time= 0.09903
Epoch: 0005 train_loss= 1.37794 time= 0.10000
Epoch: 0006 train_loss= 1.37138 time= 0.10700
Epoch: 0007 train_loss= 1.36183 time= 0.10792
Epoch: 0008 train_loss= 1.34653 time= 0.10339
Epoch: 0009 train_loss= 1.32501 time= 0.10248
Epoch: 0010 train_loss= 1.29683 time= 0.11500
Epoch: 0011 train_loss= 1.26388 time= 0.10601
Epoch: 0012 train_loss= 1.23587 time= 0.10239
Epoch: 0013 train_loss= 1.21068 time= 0.10708
Epoch: 0014 train_loss= 1.19284 time= 0.10792
Epoch: 0015 train_loss= 1.18820 time= 0.11800
Epoch: 0016 train_loss= 1.14916 time= 0.12001
Epoch: 0017 train_loss= 1.14050 time= 0.11142
Epoch: 0018 train_loss= 1.12870 time= 0.08719
Epoch: 0019 train_loss= 1.12097 time= 0.09894
Epoch: 0020 train_loss= 1.10809 time= 0.10

Epoch: 0179 train_loss= 0.48355 time= 0.10100
Epoch: 0180 train_loss= 0.50985 time= 0.11099
Epoch: 0181 train_loss= 0.51213 time= 0.11496
Epoch: 0182 train_loss= 0.49460 time= 0.11402
Epoch: 0183 train_loss= 0.49745 time= 0.11698
Epoch: 0184 train_loss= 0.50504 time= 0.11295
Epoch: 0185 train_loss= 0.48433 time= 0.10867
Epoch: 0186 train_loss= 0.49070 time= 0.11726
Epoch: 0187 train_loss= 0.49021 time= 0.11306
Epoch: 0188 train_loss= 0.50332 time= 0.11076
Epoch: 0189 train_loss= 0.49928 time= 0.11296
Epoch: 0190 train_loss= 0.50007 time= 0.11107
Epoch: 0191 train_loss= 0.49070 time= 0.11500
Epoch: 0192 train_loss= 0.48880 time= 0.12028
Epoch: 0193 train_loss= 0.47823 time= 0.11300
Epoch: 0194 train_loss= 0.48294 time= 0.11200
Epoch: 0195 train_loss= 0.48701 time= 0.12100
Epoch: 0196 train_loss= 0.48682 time= 0.11200
Epoch: 0197 train_loss= 0.48010 time= 0.10680
Epoch: 0198 train_loss= 0.48158 time= 0.10800
Epoch: 0199 train_loss= 0.48223 time= 0.10900
Epoch: 0200 train_loss= 0.48219 ti

Epoch: 0156 train_loss= 0.66934 time= 0.10071
Epoch: 0157 train_loss= 0.64881 time= 0.09992
Epoch: 0158 train_loss= 0.65309 time= 0.08444
Epoch: 0159 train_loss= 0.67137 time= 0.10445
Epoch: 0160 train_loss= 0.64191 time= 0.09411
Epoch: 0161 train_loss= 0.65713 time= 0.09646
Epoch: 0162 train_loss= 0.65099 time= 0.10243
Epoch: 0163 train_loss= 0.64925 time= 0.09790
Epoch: 0164 train_loss= 0.65858 time= 0.08487
Epoch: 0165 train_loss= 0.65517 time= 0.10161
Epoch: 0166 train_loss= 0.65785 time= 0.09775
Epoch: 0167 train_loss= 0.64171 time= 0.08407
Epoch: 0168 train_loss= 0.64603 time= 0.09896
Epoch: 0169 train_loss= 0.64038 time= 0.08916
Epoch: 0170 train_loss= 0.66174 time= 0.09178
Epoch: 0171 train_loss= 0.63229 time= 0.11166
Epoch: 0172 train_loss= 0.66407 time= 0.08688
Epoch: 0173 train_loss= 0.66163 time= 0.08061
Epoch: 0174 train_loss= 0.64289 time= 0.10833
Epoch: 0175 train_loss= 0.65062 time= 0.09493
Epoch: 0176 train_loss= 0.65701 time= 0.07379
Epoch: 0177 train_loss= 0.65629 ti

Epoch: 0132 train_loss= 0.53634 time= 0.09789
Epoch: 0133 train_loss= 0.53904 time= 0.08229
Epoch: 0134 train_loss= 0.53084 time= 0.08566
Epoch: 0135 train_loss= 0.53118 time= 0.09904
Epoch: 0136 train_loss= 0.53865 time= 0.09671
Epoch: 0137 train_loss= 0.54059 time= 0.08862
Epoch: 0138 train_loss= 0.53778 time= 0.11223
Epoch: 0139 train_loss= 0.54327 time= 0.09804
Epoch: 0140 train_loss= 0.54059 time= 0.09039
Epoch: 0141 train_loss= 0.53780 time= 0.09930
Epoch: 0142 train_loss= 0.53221 time= 0.09785
Epoch: 0143 train_loss= 0.53537 time= 0.07877
Epoch: 0144 train_loss= 0.53776 time= 0.08426
Epoch: 0145 train_loss= 0.53253 time= 0.09321
Epoch: 0146 train_loss= 0.51712 time= 0.09216
Epoch: 0147 train_loss= 0.53239 time= 0.09037
Epoch: 0148 train_loss= 0.52657 time= 0.09549
Epoch: 0149 train_loss= 0.52476 time= 0.08912
Epoch: 0150 train_loss= 0.52733 time= 0.08304
Epoch: 0151 train_loss= 0.53117 time= 0.09352
Epoch: 0152 train_loss= 0.53305 time= 0.09632
Epoch: 0153 train_loss= 0.51740 ti

Epoch: 0108 train_loss= 0.60346 time= 0.10529
Epoch: 0109 train_loss= 0.58610 time= 0.09980
Epoch: 0110 train_loss= 0.61358 time= 0.09117
Epoch: 0111 train_loss= 0.57018 time= 0.09383
Epoch: 0112 train_loss= 0.57668 time= 0.09969
Epoch: 0113 train_loss= 0.58144 time= 0.10983
Epoch: 0114 train_loss= 0.59275 time= 0.10777
Epoch: 0115 train_loss= 0.57258 time= 0.10173
Epoch: 0116 train_loss= 0.58660 time= 0.09953
Epoch: 0117 train_loss= 0.57177 time= 0.09868
Epoch: 0118 train_loss= 0.58869 time= 0.09877
Epoch: 0119 train_loss= 0.56769 time= 0.09910
Epoch: 0120 train_loss= 0.56516 time= 0.10192
Epoch: 0121 train_loss= 0.56504 time= 0.09665
Epoch: 0122 train_loss= 0.56254 time= 0.09034
Epoch: 0123 train_loss= 0.56712 time= 0.09943
Epoch: 0124 train_loss= 0.55296 time= 0.11446
Epoch: 0125 train_loss= 0.55868 time= 0.09876
Epoch: 0126 train_loss= 0.55820 time= 0.10420
Epoch: 0127 train_loss= 0.56529 time= 0.09680
Epoch: 0128 train_loss= 0.54340 time= 0.10236
Epoch: 0129 train_loss= 0.55532 ti

...

Epoch: 0038 train_loss= 0.84553 time= 0.11800
Epoch: 0039 train_loss= 0.85354 time= 0.10038
Epoch: 0040 train_loss= 0.82522 time= 0.09441
Epoch: 0041 train_loss= 0.83439 time= 0.09864
Epoch: 0042 train_loss= 0.85353 time= 0.09302
Epoch: 0043 train_loss= 0.84148 time= 0.11993
Epoch: 0044 train_loss= 0.79213 time= 0.11200
Epoch: 0045 train_loss= 0.84542 time= 0.10700
Epoch: 0046 train_loss= 0.79003 time= 0.09600
Epoch: 0047 train_loss= 0.79532 time= 0.09379
Epoch: 0048 train_loss= 0.78551 time= 0.09800
Epoch: 0049 train_loss= 0.79003 time= 0.09700
Epoch: 0050 train_loss= 0.77444 time= 0.09601
Epoch: 0051 train_loss= 0.78162 time= 0.09499
Epoch: 0052 train_loss= 0.77400 time= 0.09395
Epoch: 0053 train_loss= 0.75808 time= 0.09651
Epoch: 0054 train_loss= 0.75916 time= 0.09979
Epoch: 0055 train_loss= 0.73812 time= 0.09900
Epoch: 0056 train_loss= 0.74302 time= 0.11758
Epoch: 0057 train_loss= 0.73818 time= 0.11079
Epoch: 0058 train_loss= 0.74433 time= 0.09391
Epoch: 0059 train_loss= 0.73432 ti

Epoch: 0014 train_loss= 1.18701 time= 0.10130
Epoch: 0015 train_loss= 1.16974 time= 0.08811
Epoch: 0016 train_loss= 1.15985 time= 0.09153
Epoch: 0017 train_loss= 1.13033 time= 0.08757
Epoch: 0018 train_loss= 1.11890 time= 0.08923
Epoch: 0019 train_loss= 1.10686 time= 0.10063
Epoch: 0020 train_loss= 1.09911 time= 0.10085
Epoch: 0021 train_loss= 1.07635 time= 0.09470
Epoch: 0022 train_loss= 1.07402 time= 0.10153
Epoch: 0023 train_loss= 1.05927 time= 0.09789
Epoch: 0024 train_loss= 1.04298 time= 0.10226
Epoch: 0025 train_loss= 1.04239 time= 0.09773
Epoch: 0026 train_loss= 1.02908 time= 0.10214
Epoch: 0027 train_loss= 1.00987 time= 0.10079
Epoch: 0028 train_loss= 1.00330 time= 0.10349
Epoch: 0029 train_loss= 1.00387 time= 0.09852
Epoch: 0030 train_loss= 0.98110 time= 0.10008
Epoch: 0031 train_loss= 0.97237 time= 0.10158
Epoch: 0032 train_loss= 0.95123 time= 0.10715
Epoch: 0033 train_loss= 0.94222 time= 0.09436
Epoch: 0034 train_loss= 0.94462 time= 0.09642
Epoch: 0035 train_loss= 0.91254 ti

Epoch: 0194 train_loss= 0.55696 time= 0.09278
Epoch: 0195 train_loss= 0.54800 time= 0.10324
Epoch: 0196 train_loss= 0.54550 time= 0.10265
Epoch: 0197 train_loss= 0.54409 time= 0.09700
Epoch: 0198 train_loss= 0.55028 time= 0.09629
Epoch: 0199 train_loss= 0.54590 time= 0.09700
Epoch: 0200 train_loss= 0.53613 time= 0.10401
Testing model...
Masking test edges...
torch.Size([4230, 4230]) torch.Size([4230, 4230])
Training...
Epoch: 0001 train_loss= 1.38632 time= 0.10400
Epoch: 0002 train_loss= 1.38567 time= 0.10104
Epoch: 0003 train_loss= 1.38363 time= 0.10195
Epoch: 0004 train_loss= 1.38077 time= 0.10400
Epoch: 0005 train_loss= 1.37590 time= 0.09600
Epoch: 0006 train_loss= 1.36786 time= 0.10000
Epoch: 0007 train_loss= 1.35594 time= 0.09800
Epoch: 0008 train_loss= 1.33680 time= 0.09800
Epoch: 0009 train_loss= 1.31029 time= 0.10241
Epoch: 0010 train_loss= 1.27610 time= 0.09598
Epoch: 0011 train_loss= 1.25064 time= 0.10700
Epoch: 0012 train_loss= 1.21885 time= 0.09928
Epoch: 0013 train_loss= 1

Epoch: 0172 train_loss= 0.76829 time= 0.09325
Epoch: 0173 train_loss= 0.75066 time= 0.09475
Epoch: 0174 train_loss= 0.76220 time= 0.09300
Epoch: 0175 train_loss= 0.76743 time= 0.10205
Epoch: 0176 train_loss= 0.76577 time= 0.09500
Epoch: 0177 train_loss= 0.78031 time= 0.09101
Epoch: 0178 train_loss= 0.78783 time= 0.09894
Epoch: 0179 train_loss= 0.79823 time= 0.09505
Epoch: 0180 train_loss= 0.79023 time= 0.09301
Epoch: 0181 train_loss= 0.80164 time= 0.09366
Epoch: 0182 train_loss= 0.80502 time= 0.10100
Epoch: 0183 train_loss= 0.79967 time= 0.09484
Epoch: 0184 train_loss= 0.78567 time= 0.09517
Epoch: 0185 train_loss= 0.77359 time= 0.08597
Epoch: 0186 train_loss= 0.77840 time= 0.09955
Epoch: 0187 train_loss= 0.76671 time= 0.09535
Epoch: 0188 train_loss= 0.78097 time= 0.09698
Epoch: 0189 train_loss= 0.76932 time= 0.09700
Epoch: 0190 train_loss= 0.76830 time= 0.09900
Epoch: 0191 train_loss= 0.76563 time= 0.09601
Epoch: 0192 train_loss= 0.77896 time= 0.09499
Epoch: 0193 train_loss= 0.76832 ti

Epoch: 0149 train_loss= 0.77395 time= 0.08549
Epoch: 0150 train_loss= 0.75884 time= 0.10151
Epoch: 0151 train_loss= 0.77678 time= 0.09979
Epoch: 0152 train_loss= 0.78461 time= 0.09394
Epoch: 0153 train_loss= 0.77067 time= 0.09864
Epoch: 0154 train_loss= 0.76103 time= 0.09620
Epoch: 0155 train_loss= 0.77918 time= 0.09380
Epoch: 0156 train_loss= 0.76999 time= 0.09422
Epoch: 0157 train_loss= 0.79425 time= 0.10000
Epoch: 0158 train_loss= 0.80093 time= 0.09260
Epoch: 0159 train_loss= 0.81885 time= 0.10400
Epoch: 0160 train_loss= 0.80242 time= 0.11100
Epoch: 0161 train_loss= 0.79364 time= 0.09274
Epoch: 0162 train_loss= 0.77229 time= 0.09112
Epoch: 0163 train_loss= 0.76766 time= 0.09089
Epoch: 0164 train_loss= 0.78976 time= 0.09520
Epoch: 0165 train_loss= 0.78958 time= 0.09779
Epoch: 0166 train_loss= 0.79208 time= 0.10131
Epoch: 0167 train_loss= 0.76706 time= 0.09700
Epoch: 0168 train_loss= 0.76972 time= 0.09500
Epoch: 0169 train_loss= 0.79606 time= 0.09453
Epoch: 0170 train_loss= 0.77958 ti